In [52]:
import panel as pn
import altair as alt
from altair import datum
import pandas as pd
from vega_datasets import data
import datetime as dt
import hvplot.pandas
import numpy as np

pn.extension('tabulator')
pn.extension(comms='ipywidgets')


In [53]:
alt.renderers.enable('default')
pn.extension('vega')

In [54]:
year_slider=pn.widgets.IntSlider(name='Year slider',start=1950, end=2020, step=2,value=2008)
year_slider
co2_rb = pn.widgets.RadioButtonGroup(name='CO2 rb',options=['co2','co2_per_capita'],button_type='success')
co2_rb

BokehModel(combine_events=True, render_bundle={'docs_json': {'467fedc8-02d0-4a90-9917-75b8a32c290c': {'defs': …

In [55]:
#source = data.stocks()
#source = pd.DataFrame(source)
source =pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv')
source.columns


Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

In [67]:
pigs=['United States','China','India','Russia','Japan','Germany','Iran','Saudi Arabia','South Korea']
continents=['World','Asia','Oceania','Europe','Africa','North America','South America','Antarctica']

source.fillna(0)
countries=source['country'].unique().tolist()
source['co2_per_cap100k']=np.where(source.population !=0,source.co2/source.population*10e5,0)
idf_full=source.interactive()
# print(countries)
ticker = pn.widgets.Select(name='Country', options=countries)

idf_sub = idf_full[(idf_full.country.isin(pigs)) & (idf_full.year > year_slider)]

# china_df = source[source.country == 'China']
# usa_df.head()
# chart = alt.Chart(usa_df).mark_line().encode(x='year', y='co2',      
#         tooltip=alt.Tooltip(['year','co2']))
# chart

cusa = idf_sub.hvplot(x='year',y=co2_rb,by='country',title='CO2 Emission').opts(framewise=True) 
cusa
#ticker

BokehModel(combine_events=True, render_bundle={'docs_json': {'08b4fcc1-1c33-435d-b2b7-0b2aad7f0ac8': {'defs': …

In [68]:
idf_sub=idf_full[(idf_full['year']==year_slider) & (idf_full.country.isin(pigs))].groupby(['country','year'])[co2_rb].mean().to_frame().reset_index().sort_values(by='year').reset_index(drop=True)

In [70]:
idf_sub.head()
co2_table=idf_sub.pipe(pn.widgets.Tabulator, pagination='remote',page_size=10,sizing_mode='stretch_width')
co2_table

BokehModel(combine_events=True, render_bundle={'docs_json': {'d68ce935-a0fd-485e-8435-5b2d67beb90f': {'defs': …

In [74]:
dashboard = pn.Row(
    pn.Column(year_slider,co2_rb,co2_table.panel()),
    pn.Column(cusa.panel(),)
)
dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'465e22c3-8b34-41b3-9b47-02c742daed03': {'defs': …

In [25]:
# go back to the source dataframe and calculate gdp/capita

idf_sub = idf_full[(idf_full.country.isin(pigs)) & (idf_full.year > year_slider)]
co2_per_cap_plot = idf_sub.hvplot(x='year',y='co2_per_cap100k',by='country',title='CO2 per 100k Persons').opts(framewise=True) 
co2_per_cap_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'1ff52bfd-9a18-4249-8911-a1c3dd12a474': {'defs': …

In [69]:
# create list of company names (tickers) to use as options
tickers = ['AAPL', 'GOOG', 'IBM', 'MSFT']
# this creates the dropdown widget
ticker = pn.widgets.Select(name='Company', options=tickers)


In [24]:
title='Stock Price Dashboard'
subtitle='Dis and vis of stock price of selected company'

In [23]:
# this creates the date range slider
date_range_slider = pn.widgets.DateRangeSlider(
name='Date Range Slider',
start=dt.datetime(2001, 1, 1), end=dt.datetime(2010, 1, 1),
value=(dt.datetime(2001, 1, 1), dt.datetime(2010, 1, 1))
)
# date_range_slider

In [25]:
@pn.depends(ticker.param.value, date_range_slider.param.value)
def get_plot(ticker, date_range):
     # Load and format the data
     df = source # define df
     df['date'] = pd.to_datetime(df['date'])
     # create date filter using values from the range slider
     # store the first and last date range slider value in a var
     start_date = date_range_slider.value[0] 
     end_date = date_range_slider.value[1] 
     # create filter mask for the dataframe
     mask = (df['date']> start_date) & (df['date'] <= end_date)
     df = df.loc[mask] # filter the dataframe
     # create the Altair chart object
     chart = alt.Chart(df).mark_line().encode(x='date', y='price',      
        tooltip=alt.Tooltip(['date','price'])).transform_filter(
        (datum.symbol == ticker) # this ties in the filter 
        )
     return chart

In [26]:
dashboard = pn.Row(pn.Column(title, subtitle, ticker, date_range_slider),
    get_plot) # our draw chart function!


In [27]:
dashboard.servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Select(name='Company', options=['AAPL', 'GOOG', ...], value='AAPL')
        [3] DateRangeSlider(end=datetime.datetime(2010, ..., name='Date Range Slider', start=datetime.datetime(2001, ..., value=(datetime.datetime(2001, ..., value_end=datetime.datetime(2010, ..., value_start=datetime.datetime(2001, ...)
    [1] ParamFunction(function, _pane=Vega)